In [1]:
import numpy as np
import pandas as pd 

class K_means(object):
    def __init__(self,k):
        self.k = k
    def fit(self, x):
        self.x = np.array(x).astype(float)
        length= len(x)
        center_index=np.random.choice(length, self.k, replace=False)
        self.cluster_centers_=np.array([self.x[i] for i in center_index])
        init=[0]*length
        check=True
        while check :
            record = []
            for i in range(length):
                dist = [np.linalg.norm(x[i]-j) for j in self.cluster_centers_]
                record.append(np.argmin(dist))
            data= np.concatenate((x, np.array(record).reshape(-1,1)), axis=1)
            data= pd.DataFrame(data)
            self.cluster_centers_ =np.array(data.groupby(data.columns[-1])[data.columns[:-1]].mean().values)
            
            if record == init:
                check =False
                self.labels_=record
                return self
            else:
                check =True
                init =record
    def predict(self, predict_data):
        record = []
        for i in range(len(predict_data)):
            dist = [np.linalg.norm(predict_data[i]-j) for j in self.cluster_centers_]
            record.append(np.argmin(dist))
        return np.array(record)

In [2]:
from sklearn.datasets import make_blobs
from sklearn import datasets
from sklearn import preprocessing
#import matplotlib.pyplot as plt
#data = make_blobs(n_samples=200, n_features=2, centers=4, cluster_std=0.8,random_state=101)
iris = datasets.load_iris()
x=iris["data"]
y=iris["target"]
x= preprocessing.scale(x)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [8]:
#k_means = K_means(k=4).fit(data[0])
k_means=K_means(k=3).fit(x)
label=k_means.labels_

acc=0
for i in range(0,150,50):
    count=pd.value_counts(label[i:i+50])
    acc+=count.iloc[0]
acc=acc/150
print(acc)


0.8333333333333334


In [89]:
#plt.scatter(data[0][:,0],data[0][:,1],c=k_means.labels_,cmap='rainbow')
#plt.show()

In [7]:
CROSS_RATE = 0.45                  # mating probability (DNA crossover)
MUTATION_RATE = 0.003              # mutation probability
POP_SIZE = 800                     # population size
N_GENERATIONS = 9100

class GA(object):
    def __init__(self, DNA_size, cross_rate, mutation_rate, pop_size, K,data):
        self.cross_rate = cross_rate
        self.mutate_rate = mutation_rate
        self.pop_size = pop_size
        self.DNA_size = DNA_size
        self.pop = np.random.randint(0,K, size=(pop_size, DNA_size))  
        self.data = data
        self.K = K

    def translateDNA(self, DNA):                 # convert to readable string
        return DNA

    def compute_d2(self, X, centers):
        new=np.zeros((X.shape[0],centers.shape[0]))
    
        for i in range(len(centers)):
            new[:,i]=np.sum((X-centers[i])**2,axis=1)#(X[:,0]-centers[i][0])**2+(X[:,1]-centers[i][1])**2
        return new
    
    def get_fitness(self):                   
        result =[]
        for i in range(len(self.pop)):
            cur = self.pop[i]
            combine = np.concatenate((self.data, np.array(cur).reshape(-1,1)), axis=1)
            combine = pd.DataFrame(combine)
            centers = np.array(combine.groupby(combine.columns[-1])[combine.columns[:-1]].mean().values)
            
            d_matrix = self.compute_d2(self.data,centers)
            WCSS = (np.amin(d_matrix, axis=1)).sum()
            result.append(WCSS)
        return np.array(result)

    def select(self):
        fitness = self.get_fitness()      # add a small amount to avoid all zero fitness

        inverse_probability = [1/x for x in fitness]

        idx = np.random.choice(np.arange(self.pop_size), size=self.pop_size, replace=True, p=inverse_probability/sum(inverse_probability))     
        return self.pop[idx]

    def crossover(self, parent, pop):
        if np.random.rand() < self.cross_rate:
            i_ = np.random.randint(0, self.pop_size, size=1)                        # select another individual from pop
            cross_points = np.random.randint(0, 2, self.DNA_size).astype(np.bool)   # choose crossover points
            parent[cross_points] = pop[i_, cross_points]                            # mating and produce one child
        return parent

    def mutate(self, child):
        for point in range(self.DNA_size):
            if np.random.rand() < self.mutate_rate:
                child[point] = np.random.randint(0,self.K)  # choose a random ASCII index
        return child

    def evolve(self):
        pop = self.select()
        pop_copy = pop.copy()
        for parent in pop:  # for every parent
            child = self.crossover(parent, pop_copy)
            child = self.mutate(child)
            parent[:] = child
        self.pop = pop


ga = GA(DNA_size=len(data[0]),cross_rate=CROSS_RATE,
    mutation_rate=MUTATION_RATE, pop_size=POP_SIZE,K=4,data=np.array(data[0]))

In [90]:
#for generation in range(N_GENERATIONS):
 #   fitness = ga.get_fitness()
 #   best_DNA = ga.pop[np.argmax(fitness)]
 #   best = ga.translateDNA(best_DNA)
 #   if generation%3000==0:
 #       plt.scatter(data[0][:,0],data[0][:,1],c=best,cmap='rainbow')
 #       plt.show()
 #   ga.evolve()